In [17]:
%matplotlib inline

import glob
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.utils.exceptions import AstropyWarning, AstropyUserWarning
from scipy.sparse.linalg import inv
import warnings

from lsst.eotest.fitsTools import fitsWriteto
import lsst.eotest.image_utils as imutils
from lsst.eotest.sensor.MaskedCCD import MaskedCCD
from ctisim.matrix import cti_operator


In [20]:
## Task input variables
outfile = 'test.fits'
cti_results = {i: 1.E-6 for i in range(1, 17)}
bias_frame = None
imfile = glob.glob('/nfs/slac/g/ki/ki19/lsst/snyder18/LSST/Data/Sim/BF_Flats/*.fits')[20]
gains = None

print(infile)
ccd = MaskedCCD(infile, bias_frame=bias_frame)

/nfs/slac/g/ki/ki19/lsst/snyder18/LSST/Data/Sim/BF_Flats/lsst_a_5000031_R20_S02_i_processed.fits


In [21]:
hdulist = fits.HDUList()
with fits.open(infile) as template:
    hdulist.append(template[0])
    hdulist[0].header['ORIGFILE'] = hdulist[0].header['FILENAME']
    hdulist[0].header['FILENAME'] = outfile
    
    for amp in imutils.allAmps(imfile):
        
        imarr = ccd.bias_subtracted_image(amp).getImage().getArray()
        if gains is not None:
            imarr *= gains[amp]
            
        ## CTI correction
        D_cti = cti_operator(cti_results[amp], imarr.shape[1])
        Dinv_cti = inv(D_cti)
        corrected_imarr = np.zeros(imarr.shape)
        for i in range(imarr.shape[0]):
    
            corrected_imarr[i, :] = Dinv_cti*imarr[i, :]

        if gains is not None:
            corrected_imarr /= gains[amp]
        hdulist.append(fits.ImageHDU(data=corrected_imarr,
                                     header=template[amp].header))
        with warnings.catch_warnings():
            for warning in (UserWarning, AstropyWarning,
                            AstropyUserWarning):
                warnings.filterwarnings('ignore', category=warning,
                                            append=True)
            fitsWriteto(hdulist, outfile, overwrite=True)

/nfs/slac/g/ki/ki19/lsst/snyder18/software/anaconda3/lib/python3.7/site-packages/scipy/sparse/linalg/dsolve/linsolve.py:133: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)
/nfs/slac/g/ki/ki19/lsst/snyder18/software/anaconda3/lib/python3.7/site-packages/scipy/sparse/linalg/dsolve/linsolve.py:203: SparseEfficiencyWarning: spsolve is more efficient when sparse b is in the CSC matrix format
  'is in the CSC matrix format', SparseEfficiencyWarning)
/nfs/slac/g/ki/ki19/lsst/snyder18/software/anaconda3/lib/python3.7/site-packages/scipy/sparse/linalg/dsolve/linsolve.py:133: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)
/nfs/slac/g/ki/ki19/lsst/snyder18/software/anaconda3/lib/python3.7/site-packages/scipy/sparse/linalg/dsolve/linsolve.py:203: SparseEfficiencyWarning: spsolve is more efficient when sparse b is in the CSC matrix format
  'is in the CSC matrix format', SparseEfficiencyWarning)
